<a href="https://colab.research.google.com/github/KyoldFusion/StrangerThings-IMDB-Netflix-/blob/main/NetflixIMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa

In [3]:
 #import packages

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType, FloatType
from pyspark.sql.functions import col, unix_timestamp, to_date
from pyspark.sql.functions import to_timestamp
from datetime import timedelta
from pyspark.sql.functions import udf

# we are going to use this to time our queries.
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
ratings = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/ratings.csv"
spark.sparkContext.addFile(ratings)
rf = spark.read.csv(SparkFiles.get("ratings.csv"), sep=",", header=True)

In [5]:
#Create Temp View for ratings.csv
rf.createOrReplaceTempView('ratings')

In [6]:
# Read in data from S3 Bucket
basics = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/titles_basic.csv"
spark.sparkContext.addFile(basics)
bf = spark.read.csv(SparkFiles.get("titles_basic.csv"), sep=",", header=True)
#bf = bf.withColumn('runtimeminutes', col('runtimeminutes').cast('float'))

In [7]:
#Create Temp View for title_basics.csv
bf.createOrReplaceTempView('basics')

In [8]:
# Read in data from S3 Bucket
names = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/filterNames.csv"
spark.sparkContext.addFile(names)
nf = spark.read.csv(SparkFiles.get("filterNames.csv"), sep=",", header=True)

In [9]:
#Create temp view for filterNames.csv
nf.createOrReplaceTempView('names')

In [10]:
#Read in data from S3 bucket
principals = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/filteredPrincipals.csv"
spark.sparkContext.addFile(principals)
pf = spark.read.csv(SparkFiles.get("filteredPrincipals.csv"), sep=",", header=True)

In [11]:
#Create temp view for FilteredPrincipals.csv
pf.createOrReplaceTempView('principals')

In [14]:
#Question 1 No Parquet
start_time = time.time()
spark.sql("Select DISTINCT nconst, primaryname, birthyear, deathyear, CASE WHEN birthyear IS NOT NULL and deathyear IS NOT NULL then CAST(deathyear - birthyear AS decimal(3,0)) ELSE COALESCE((birthyear - deathyear), 2021-birthyear) END  Age, CASE WHEN deathyear IS NULL AND birthyear IS NOT NULL THEN 'alive' WHEN deathyear IS NOT NULL THEN 'deceased' WHEN birthyear IS NULL AND deathyear is NULL THEN 'unknown' END livingstatus, CASE WHEN deathyear IS NULL THEN true WHEN deathyear IS NOT NULL THEN false END isalive from names where (primaryprofession LIKE '%actor%' OR primaryprofession LIKE '%actress%') AND birthyear IS NOT NULL").show()
print("---- %s Seconds ---" % (time.time() - start_time))

+---------+--------------------+---------+---------+----+------------+-------+
|   nconst|         primaryname|birthyear|deathyear| Age|livingstatus|isalive|
+---------+--------------------+---------+---------+----+------------+-------+
|nm1446700|          Ben Graves|     1974|     null|47.0|       alive|   true|
|nm3128055|Gonzalo Sánchez S...|     1996|     null|25.0|       alive|   true|
|nm3129672|       Rufus Hegeman|     1982|     null|39.0|       alive|   true|
|nm3130003|Katharina M. Schu...|     1977|     null|44.0|       alive|   true|
|nm1450306|      Bernard Kierat|     1976|     null|45.0|       alive|   true|
|nm0464793|        László Komár|     1944|     2012|68.0|    deceased|  false|
|nm3135124|   Tessie Richardson|     1990|     null|31.0|       alive|   true|
|nm3129534|     Steffen Dziczek|     1982|     null|39.0|       alive|   true|
|nm3131295|        Xanthe Flint|     1997|     null|24.0|       alive|   true|
|nm8467502|      Don Vinkenberg|     1997|     null|

In [15]:
#Writing parquet into files
nf.write.parquet('parquet_names', mode='overwrite')

In [16]:
#Read parquet created
npf = spark.read.parquet('parquet_names')

In [17]:
#create temp parquet view for names
npf.createOrReplaceTempView('pnames')

In [ ]:
####################### QUESTION SEGMENT 1 ####################

In [18]:
#Question 1 Parquet
start_time = time.time()
spark.sql("Select DISTINCT nconst, primaryname, birthyear, deathyear, CASE WHEN birthyear IS NOT NULL and deathyear IS NOT NULL then CAST(deathyear - birthyear AS decimal(3,0)) ELSE COALESCE((birthyear - deathyear), 2021-birthyear) END  Age, CASE WHEN deathyear IS NULL AND birthyear IS NOT NULL THEN 'alive' WHEN deathyear IS NOT NULL THEN 'deceased' WHEN birthyear IS NULL AND deathyear is NULL THEN 'unknown'  END LivingStatus, CASE WHEN deathyear IS NULL THEN true WHEN deathyear IS NOT NULL THEN false END isalive from pnames where (primaryprofession LIKE '%actor%' OR primaryprofession LIKE '%actress%') AND birthyear IS NOT NULL").show()
print("---- %s Seconds ---" % (time.time() - start_time))

+---------+--------------------+---------+---------+----+------------+-------+
|   nconst|         primaryname|birthyear|deathyear| Age|LivingStatus|isalive|
+---------+--------------------+---------+---------+----+------------+-------+
|nm0961445|    The Naked Cowboy|     1970|     null|51.0|       alive|   true|
|nm0962261|           Will Kemp|     1977|     null|44.0|       alive|   true|
|nm2598507|    Francys Sudnicka|     1979|     null|42.0|       alive|   true|
|nm0988999|       Fulton Burley|     1922|     2007|85.0|    deceased|  false|
|nm0989397|Smriti Malhotra-I...|     1976|     null|45.0|       alive|   true|
|nm2635396|      Ryan Conferido|     1983|     null|38.0|       alive|   true|
|nm3799680|        Eva Sunshine|     1986|     null|35.0|       alive|   true|
|nm1004333|       Ilyssa Fradin|     1967|     null|54.0|       alive|   true|
|nm3810545|    Amila Terzimehic|     1988|     null|33.0|       alive|   true|
|nm1088648|          Lara Kelly|     1979|     null|

In [ ]:
############################# QUESTION SEGMENT 2 ######################

In [ ]:
#Conversion Function
def minutesToHourMinutes(minuteString):
  if minuteString:
    minuteAsFloat = float(minuteString)
    return str(timedelta(minutes=minuteAsFloat))[:-3]
  else:
    return "0:0"

minutesToHourMinutesUdf = udf(lambda z: minutesToHourMinutes(z), StringType())
spark.udf.register("minutesToHourMinutesUdf", minutesToHourMinutesUdf)

<function __main__.<lambda>>

In [ ]:
from datetime import timedelta

str(timedelta(minutes=700))

'11:40:00'

In [ ]:
a = [1,2,3,4,5,6,7]
a[:-3]

[1, 2, 3, 4]

In [ ]:
#Question 2 no parquet
start_time = time.time()
spark.sql("""Select b.originaltitle, r.averagerating, b.titletype, COALESCE(minutesToHourMinutesUdf(b.runtimeminutes), 0)
         runtime, b.genres from ratings r join basics b ON b.tconst = r.tconst""").show()
print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+-------------+---------+-------+--------------------+
|       originaltitle|averagerating|titletype|runtime|              genres|
+--------------------+-------------+---------+-------+--------------------+
|           500 Almas|          7.2|    movie|   1:45|         Documentary|
|         A1 tou tiao|          5.9|    movie|   1:35|    Mystery,Thriller|
| AD/BC: A Rock Opera|          7.4|  tvMovie|   0:30|      Comedy,Musical|
|       Ah ma yau nan|          5.5|    movie|   1:33|        Comedy,Crime|
|      Bau lit do see|          5.4|    movie|   1:39|              Action|
|  The Band Aid Story|          8.1|  tvMovie|   1:35|         Documentary|
|A Beachcombers Ch...|          7.0|  tvMovie|   2:00|Adventure,Comedy,...|
|Blink 182: Punk P...|          4.1|    video|   1:01|         Documentary|
|Boogie special: 5...|          1.0|  tvMovie|   0:26|   Documentary,Music|
|Boogie special: M...|          7.7|  tvMovie|   0:28|Biography,Documen...|
|The British

In [ ]:
rf.write.parquet('parquet_ratings', mode='overwrite')

In [ ]:
rpf = spark.read.parquet('parquet_ratings')

In [ ]:
rpf.createOrReplaceTempView('rratings')

In [ ]:
bf.write.parquet('parquet_basics', mode='overwrite')

In [ ]:
bpf = spark.read.parquet('parquet_basics')

In [ ]:
bpf.createOrReplaceTempView('bbasics')

In [ ]:
#Writing parquet into files
pf.write.parquet('parquet_principals', mode='overwrite')

In [ ]:
#Read parquet created
ppf = spark.read.parquet('parquet_principals')

In [ ]:
#create temp parquet view for prinicpals
ppf.createOrReplaceTempView('pprincipals')

In [ ]:
#Question 2 Parquet
start_time = time.time()
spark.sql("""Select b.originaltitle, r.averagerating, b.titletype, COALESCE(minutesToHourMinutesUdf
(b.runtimeminutes), 0) runtime, b.genres from rratings r join bbasics b ON b.tconst = r.tconst""").show()
print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+-------------+---------+-------+--------------------+
|       originaltitle|averagerating|titletype|runtime|              genres|
+--------------------+-------------+---------+-------+--------------------+
|           500 Almas|          7.2|    movie|   1:45|         Documentary|
|         A1 tou tiao|          5.9|    movie|   1:35|    Mystery,Thriller|
| AD/BC: A Rock Opera|          7.4|  tvMovie|   0:30|      Comedy,Musical|
|       Ah ma yau nan|          5.5|    movie|   1:33|        Comedy,Crime|
|      Bau lit do see|          5.4|    movie|   1:39|              Action|
|  The Band Aid Story|          8.1|  tvMovie|   1:35|         Documentary|
|A Beachcombers Ch...|          7.0|  tvMovie|   2:00|Adventure,Comedy,...|
|Blink 182: Punk P...|          4.1|    video|   1:01|         Documentary|
|Boogie special: 5...|          1.0|  tvMovie|   0:26|   Documentary,Music|
|Boogie special: M...|          7.7|  tvMovie|   0:28|Biography,Documen...|
|The British

In [ ]:
########### QUESTION SEGMENT 3 #####################
233677
7544510
8595190

In [ ]:
233677+7544510

7778187

In [ ]:
spark.sql("SELECT COUNT(*) FROM names").show()

+--------+
|count(1)|
+--------+
| 8595190|
+--------+



In [ ]:
#Question #3
start_time = time.time()
spark.sql("Select n.primaryname, b.primarytitle, n.knownFor, n.primaryprofession from names n join basics b ON n.knownFor = b.tconst WHERE n.primaryprofession ='producer'").show()
print('--- %s Seconds ---' % (time.time() - start_time))

+-------------------+--------------------+---------+-----------------+
|        primaryname|        primarytitle| knownFor|primaryprofession|
+-------------------+--------------------+---------+-----------------+
|      Werner Kernig|  Legion of the Dead|tt0213802|         producer|
|   Michael Okulitch|    Suspicious River|tt0219333|         producer|
|      Erik Stensrud|    Suspicious River|tt0219333|         producer|
|     Michael Warren|On Hallowed Groun...|tt0245330|         producer|
|    Dennis Brownlee|On Hallowed Groun...|tt0245330|         producer|
|   Cornelia Burnham|On Hallowed Groun...|tt0245330|         producer|
|    Philip M. Frost|     Killer Instinct|tt0246000|         producer|
|   Denis Shusterman|     Killer Instinct|tt0246000|         producer|
|      Bill Atherton|        Thank Heaven|tt0252902|         producer|
|       Luciano Lisi|       One Eyed King|tt0254626|         producer|
| Christine Kavanagh|       One Eyed King|tt0254626|         producer|
|Zulfi

In [ ]:
#Question 3 Parquet
start_time = time.time()
spark.sql("Select n.primaryname, b.primarytitle, n.knownFor, n.primaryprofession from pnames n join bbasics b ON n.knownFor = b.tconst WHERE n.primaryprofession = 'producer' ").show()
print('--- %s Seconds ---' % (time.time() - start_time))

+-----------------+--------------------+----------+-----------------+
|      primaryname|        primarytitle|  knownFor|primaryprofession|
+-----------------+--------------------+----------+-----------------+
|   Akeko Yokoyama|            Kamichu!| tt0961173|         producer|
|    Chris Shearer|  In This Short Life| tt0962765|         producer|
|     Linda Dippel|Barney: Let's Go ...| tt2591718|         producer|
|       Jay Nelson|     Beyond the Pale| tt0934392|         producer|
|        Mike Sano|        Super Tanker| tt1776309|         producer|
|     Holden Hayes|The John Kerwin Show| tt0849013|         producer|
|     Lisa Jackson|The Firm: Total B...| tt0947008|         producer|
|  Jason Bangerter|        The Downline| tt0970929|         producer|
|Jennifer Bernardi|Renovation Realities| tt1328346|         producer|
|Jennifer Bernardi|    Designed to Sell| tt0415415|         producer|
|       Lori Drake|        The Downline| tt0970929|         producer|
|    Robert Willis| 

In [ ]:
########### QUESTION SEGMENT 4 #####################

In [ ]:
#Question 4
start_time = time.time()
spark.sql("Select DISTINCT p.characters, p.nconst from principals p join names n ON p.nconst = n.nconst WHERE characters is NOT null").show()
print('--- %s Seconds ---' % (time.time() - start_time))

+--------------------+---------+
|          characters|   nconst|
+--------------------+---------+
|        "[""Self""]"|nm0000198|
|     "[""Solomon""]"|nm0000198|
|"[""Shelly Runyon...|nm0000198|
|          "[""Self""|nm0000198|
|"[""Nathaniel She...|nm0000198|
|  "[""Bob Cratchit""|nm0000198|
|    "[""Narrator""]"|nm0000198|
|"[""William Holtz...|nm0000198|
|"[""Dr. Dennett N...|nm0000198|
|"[""Herman Mankie...|nm0000198|
|"[""Jürgen Mossac...|nm0000198|
|       "[""David""]"|nm0000198|
|"[""Jackson Lamb""]"|nm0000198|
|"[""Mason Verger""]"|nm0000198|
| "[""Self (2020)""]"|nm0000198|
|          "[""GO""]"|nm0000198|
| "[""Buford Dill""]"|nm0000198|
|"[""Charlie Strom...|nm0000198|
|     "[""Dreyfus""]"|nm0000198|
|"[""Nicolas Wyatt...|nm0000198|
+--------------------+---------+
only showing top 20 rows

--- 49.49090099334717 Seconds ---


In [ ]:
#Question 4 Parquet
start_time = time.time()
spark.sql("Select DISTINCT p.characters, p.nconst from pprincipals p join pnames n ON p.nconst = n.nconst WHERE characters is NOT null").show()
print('--- %s Seconds ---' % (time.time() - start_time))

+--------------------+---------+
|          characters|   nconst|
+--------------------+---------+
|        "[""Self""]"|nm0000198|
|"[""Dr. Tyrone Br...|nm0000198|
|       "[""Lynch""]"|nm0000198|
|"[""Vladislav Duk...|nm0000198|
|"[""CJCS Charles ...|nm0000198|
|    "[""Narrator""]"|nm0000198|
|    "[""Carnegie""]"|nm0000198|
|"[""Commissioner ...|nm0000198|
|"[""George Smiley...|nm0000198|
|         "[""Tau""]"|nm0000198|
|  "[""Self - Actor""|nm0000198|
|        "[""Paul""]"|nm0000198|
|"[""Winston Churc...|nm0000198|
|"[""Self - Actor""]"|nm0000198|
|"[""Counselor Pey...|nm0000198|
|"[""Rabbi Sendak""]"|nm0000198|
|          "[""GO""]"|nm0000198|
| "[""Buford Dill""]"|nm0000198|
|"[""Charlie Strom...|nm0000198|
|     "[""Dreyfus""]"|nm0000198|
+--------------------+---------+
only showing top 20 rows

--- 10.763848066329956 Seconds ---


In [ ]:
#spark.sql("Select DISTINCT xp.characters, xp.nconst from (Select DISTINCT characters, nconst from pprincipals) xp join pnames n ON xp.nconst = n.nconst WHERE xp.characters is NOT null").show(5)